In [ ]:
import SimpleITK as sitk
import nibabel as nib
import os
import numpy as np
from glob import glob
import time
from scipy import misc
import re
import matplotlib.pyplot as plt
import pandas as pd
import cv2

def load_mhd(file_name):
    sitk_img = sitk.ReadImage(file_name)
    img = sitk.GetArrayFromImage(sitk_img)  # indexes are z,y,x (notice the ordering)
    origin = sitk_img.GetOrigin()           # x,y,z  Origin in world coordinates (mm)
    spacing = sitk_img.GetSpacing()         # spacing of voxels in world coor. (mm)
    direction = sitk_img.GetDirection()     
    return img.astype('float32'), np.asarray(origin), np.asarray(spacing), direction


def zoom(img, new_shape, new_spacing, old_spacing, origin, direction, label=False):
    sitk_img = sitk.GetImageFromArray(img)
    sitk_img.SetOrigin(origin)
    sitk_img.SetSpacing(old_spacing)
    sitk_img.SetDirection(direction)

    resample = sitk.ResampleImageFilter()
    if (label == True):
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    resample.SetOutputDirection(sitk_img.GetDirection())
    resample.SetOutputOrigin(sitk_img.GetOrigin())
    resample.SetOutputSpacing(new_spacing)
    resample.SetSize(new_shape)
    newimage = resample.Execute(sitk_img)

    return newimage

def load_dicom(case_path):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(case_path)
    reader.SetFileNames(dicom_names)
    sitk_img = reader.Execute()
    img = sitk.GetArrayFromImage(sitk_img)
    origin = sitk_img.GetOrigin()
    spacing = sitk_img.GetSpacing()
    direction = sitk_img.GetDirection()

    return img.astype('float32'), np.asarray(origin), np.asarray(spacing), direction

def load_png(case_path):
    image_paths = glob(case_path + '/*')
    img = []
#     print(image_paths)
    for path in image_paths:
        image_slice = cv2.imread(path)[:,:,0]
        img.append(image_slice)
    img = np.array(img)
    return img.astype('float32')






In [ ]:
image_folder = 'G:/Dataset/Multi-organ-Abdominal/BTCV/Official/image'
label_folder = 'G:/Dataset/Multi-organ-Abdominal/BTCV/Official/label'
label_list = glob(label_folder + '/*.nii.gz')

labels_info = []
images_info = []
for file in label_list:
    print(file)
    label = load_mhd(file)
    labels_info.append(label)
    image_path = file.replace('/label\\label','/image/img')
#     print(file.replace('/label\\label','/image/TCIA_'))
    image = load_mhd(image_path)
    images_info.append(image)

In [ ]:
# cropping_csv = pd.read_csv('G:/Dataset/Multi-organ Abdominal/TCIA+BTCV/cropping.csv')
target_path = 'G:/Dataset/Multi-organ-Abdominal/BTCV/Official/preprocessed/'

for image_info,label_info,label_fname in zip(images_info,labels_info,label_list):
    
    image_array_original, origin, spacing, direction = image_info
#     image_array = image

#     label = sitk.ReadImage(data[:-7] + '_segmentation.nii.gz')
    label_array_original = label_info[0]
    image_array = image_array_original.copy()
    label_array = label_array_original.copy()
#     print(np.unique(label_array))
    mask = np.zeros(label_array.shape)
    for c in [1,2,3,4,6,7,8,11]:
        mask += (label_array==c)
#     mask = (label_array==1) + (label_array==3) + (label_array==6)
    label_array *= mask
    print(np.unique(label_array))
    label_array[label_array==11] = 5
#     label_array[label_array==3] = 2
#     label_array[label_array==6] = 3
    Crop_limZ, Crop_limY, Crop_limX = np.where(mask == 1)
    Crop_minZ = np.min(Crop_limZ)
    Crop_maxZ = np.max(Crop_limZ)
    
    image_array = image_array[:,::-1,:]
    label_array = label_array[:,::-1,:]
    
    image_array = image_array[Crop_minZ-1:Crop_maxZ+2,:,:]
    label_array = label_array[Crop_minZ-1:Crop_maxZ+2,:,:]
    
    image_array = np.clip(image_array, -125, 275)
    
    plt.imshow(image_array[10],cmap='gray')
    plt.title('Raw')
    plt.figure()
    
    
    file_id = (os.path.basename(label_fname.split('.')[0][-4:]))
    print(file_id)
#     cropping_info = cropping_csv[(cropping_csv.publisher=='Synapse') & (cropping_csv.original_id==file_id)]
#     print(cropping_info.values.tolist()[0])
#     crop_index = cropping_info.values.tolist()[0][-6:]

#     new_image_array = image_array[:, crop_index[0]:crop_index[1],crop_index[2]:crop_index[3]]
#     new_label_array = label_array[:, crop_index[0]:crop_index[1],crop_index[2]:crop_index[3]]
    print(image_array.shape, label_array.shape)
    print(np.unique(label_array))
    
    
    
    

    old_shape_z, old_shape_y, old_shape_x = image_array.shape
    old_shape = [old_shape_x, old_shape_y, old_shape_z]
    new_shape = np.array([384, 384, old_shape_z])
    new_spacing = np.array(spacing) / (new_shape * 1.0 / np.array(old_shape))
    new_shape = new_shape.tolist()
#     direction = (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
    


    
    final_image = zoom(image_array, new_shape, new_spacing, spacing, origin, direction)
    final_label = zoom(label_array, new_shape, new_spacing, spacing, origin, direction, label=True)
    sitk.WriteImage(final_image, target_path + 'BTCV_' + str(file_id) + '.nii.gz') 
    sitk.WriteImage(final_label, target_path + 'BTCV_' + str(file_id) + '_segmentation.nii.gz')
